AMS 516 - Stochastic Volatility Option hedging

Members: 
Sahil Khaja Huzoor

Nick Koukounas

In [2]:
import torch,math
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
mu, r = 0.10, 0.02
kappa, theta, xi, rho = 2.0, 0.04, 0.30, -0.70
gamma = 2.0 # This is the risk aversion parameter under CARA utility function. For gamma > 0, the individual is risk averse, for gamma < 0, the individual is risk seeking. 

K, T = 100.0, 1.0 

#domains of stock price and volatility
X_min, X_max = -2.0*K, 2.0*K
S_min, S_max = 20.0, 400.0
v_min, v_max = 1e-4, 0.5

# sample sizes 
N_int  = 8000
N_tau0 = 2000
N_b    = 1000

# defining our payoff function

def call_payoff(S):
    return torch.clamp(S-K, min=0.0)


def sample_uniform(n, low, high):
    return low + (high - low) * torch.rand(n, 1, device=device)    
    

class FeedForwardNet(torch.nn.Module):
    def __init__(self, in_dim=4, h1=128, h2=128, h3=64):
        super().__init__()
        self.fc1 = torch.nn.Linear(in_dim, h1)
        self.fc2 = torch.nn.Linear(h1, h2)
        self.fc3 = torch.nn.Linear(h2, h3)
        self.out = torch.nn.Linear(h3, 1)
        self.act = torch.nn.Tanh()
    def forward(self, x):
        x = self.act(self.fc1(x))
        x = self.act(self.fc2(x))
        x = self.act(self.fc3(x))
        return self.out(x)

V_theta = FeedForwardNet().to(device)

opt = torch.optim.Adam(V_theta.parameters(), lr=1e-3)

def V_terminal(X, S):
    return -torch.exp(-gamma * (X - call_payoff(S)))

def V_and_grads(tau,X,S,v):
    tau.requires_grad_(True);S.requires_grad_(True);v.requires_grad_(True);X.requires_grad_(True)

    x = torch.cat([tau,X,S,v],dim = 1)
    V = V_theta(x) # My value function that is being approximated with all the parameters

    V_tau = torch.autograd.grad(V,tau,grad_outputs=torch.ones_like(V),retain_graph=True,create_graph=True)[0]
    V_X   = torch.autograd.grad(V, X,   grad_outputs=torch.ones_like(V),retain_graph=True, create_graph=True)[0]
    V_S   = torch.autograd.grad(V, S,   grad_outputs=torch.ones_like(V),retain_graph=True, create_graph=True)[0]
    V_v   = torch.autograd.grad(V, v,   grad_outputs=torch.ones_like(V),retain_graph=True, create_graph=True)[0]

    V_XX  = torch.autograd.grad(V_X, X, grad_outputs=torch.ones_like(V_X),retain_graph=True, create_graph=True)[0]
    V_SS  = torch.autograd.grad(V_S, S, grad_outputs=torch.ones_like(V_S),retain_graph=True, create_graph=True)[0]
    V_vv  = torch.autograd.grad(V_v, v, grad_outputs=torch.ones_like(V_v),retain_graph=True, create_graph=True)[0]

    V_XS  = torch.autograd.grad(V_X, S, grad_outputs=torch.ones_like(V_X),retain_graph=True, create_graph=True)[0]
    V_Xv  = torch.autograd.grad(V_X, v, grad_outputs=torch.ones_like(V_X),retain_graph=True, create_graph=True)[0]
    V_Sv  = torch.autograd.grad(V_S, v, grad_outputs=torch.ones_like(V_S),retain_graph=True, create_graph=True)[0]

    return V, V_tau, V_X, V_S, V_v, V_XX, V_SS, V_vv, V_XS, V_Xv, V_Sv

def hjb_residual(tau,X,S,v):
    V, V_tau, V_X, V_S, V_v, V_XX, V_SS, V_vv, V_XS, V_Xv, V_Sv = V_and_grads(tau, X, S, v)
    sqrtv = torch.sqrt(torch.clamp(v,min=1e-8))

    drift_X = r * X * V_X
    drift_v = kappa * (theta - v) * V_v
    diff_S  = 0.5 * v * S**2 * V_SS
    diff_v  = 0.5 * xi**2 * v * V_vv
    mix_Sv  = rho * xi * S * torch.clamp(v,min=1e-8) * V_Sv
    # These are all the deterministic parts of the HJB equation


    # This is the optimization over pi term
    num = (mu - r) * V_X + v * S * V_XS + rho * xi * v * V_Xv
    den = 2.0 * torch.clamp(v * V_XX, min=1e-10)
    control = (num**2) / den

    # Final HJB term
    R = -V_tau + drift_X + drift_v + diff_S + diff_v + mix_Sv - control
    return R, V

def boundary_loss(m=N_b): # This code is for boundary condition of Stock and volatility. We essentially say that the value function will barely change when there is a change in stock prices at low and high values of the stock price since they are either deep in the money or deep out of the money 

    # This is the Neumann boundary condition: ie value of the derivative of a function at its boundary, also known as flux

    # picking random points for time, wealth and variance inside the domain 
    tau_b = sample_uniform(m, 0.0, T)   
    X_b   = sample_uniform(m, X_min, X_max)
    v_b   = sample_uniform(m, v_min, v_max)
    # These are points along the boundary walls of our training domain

    # We are fixing Stock price to be next to its boundary values


    # Computing The slopes of V at low and high stock price boundaries
    S_lo = torch.full_like(tau_b,S_min) # Creating a tensor of just the minimum value of the stock price
    S_hi = torch.full_like(tau_b,S_max) # Creating a tensor of just the maximum value of the stock price

    _,_,_,V_S_lo,_,_,_,_,_,_,_ = V_and_grads(tau_b,X_b,S_lo,v_b)
    _,_,_,V_S_hi,_,_,_,_,_,_,_ = V_and_grads(tau_b,X_b,S_hi,v_b)

    loss_S = 1e-4*((V_S_lo**2).mean() + (V_S_hi**2).mean())


    # repeating for volatility boundary 
    tau_b2 = sample_uniform(m, 0.0, T)
    S_b2   = sample_uniform(m, S_min, S_max)
    X_b2   = sample_uniform(m, X_min, X_max)
    v_lo   = torch.full_like(tau_b2, v_min)
    v_hi   = torch.full_like(tau_b2, v_max)
    _,_,_,_,V_v_lo,_,_,_,_,_,_ = V_and_grads(tau_b2,X_b2,S_b2,v_lo)
    _,_,_,_,V_v_hi,_,_,_,_,_,_ = V_and_grads(tau_b2,X_b2,S_b2,v_hi)
    loss_v = 1e-4*((V_v_lo**2).mean() + (V_v_hi**2).mean())

    return loss_S+loss_v


def train(steps=3000):
    for it in range(steps):

        # This is the Dirichlet condition: ie value of the function at time boundary 
        tau = sample_uniform(N_int, 0.0, T)
        X   = sample_uniform(N_int, X_min, X_max)
        S   = sample_uniform(N_int, S_min, S_max)
        v   = sample_uniform(N_int, v_min, v_max)

        # Computing the PDE residual with the random points simulated

        R,_ = hjb_residual(tau, X, S, v)
        loss_pde = (R**2).mean() # mean square PDE residual

        # Terminal condition 

        # This code is for boundary condition of time 
        tau0 = torch.zeros(N_tau0, 1, device=device)
        X0   = sample_uniform(N_tau0, X_min, X_max)
        S0   = sample_uniform(N_tau0, S_min, S_max)
        v0   = sample_uniform(N_tau0, v_min, v_max)

        V_T_pred = V_theta(torch.cat([tau0,X0,S0,v0],dim=1))
        V_T_true = V_terminal(X0,S0)
        loss_term = ((V_T_pred - V_T_true)**2).mean()

        # Enforcing the boundary condition
        loss_bc = boundary_loss()

        # Total loss
        loss = loss_term + loss_bc + loss_pde


        # backpropagating the loss function

        opt.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(V_theta.parameters(),5.0)


        if it % 200 == 0:
            print(f"[{it:4d}] total={loss.item():.3e} pde={loss_pde.item():.3e} term={loss_term.item():.3e} bc={loss_bc.item():.3e}")


train(steps=3000)





     










[   0] total=inf pde=1.968e+14 term=inf bc=4.018e-09
[ 200] total=inf pde=1.713e+14 term=inf bc=4.345e-09
[ 400] total=inf pde=1.914e+14 term=inf bc=4.123e-09
[ 600] total=inf pde=2.383e+14 term=inf bc=4.194e-09
[ 800] total=inf pde=2.017e+14 term=inf bc=4.111e-09
[1000] total=inf pde=1.917e+14 term=inf bc=4.321e-09
[1200] total=inf pde=2.897e+14 term=inf bc=4.546e-09
[1400] total=inf pde=2.127e+14 term=inf bc=4.108e-09
[1600] total=inf pde=1.598e+14 term=inf bc=4.739e-09
[1800] total=inf pde=2.162e+14 term=inf bc=4.451e-09
[2000] total=inf pde=2.599e+14 term=inf bc=4.181e-09
[2200] total=inf pde=3.077e+14 term=inf bc=4.221e-09
[2400] total=inf pde=2.713e+14 term=inf bc=4.568e-09
[2600] total=inf pde=1.576e+14 term=inf bc=4.438e-09
[2800] total=inf pde=2.062e+14 term=inf bc=4.240e-09


Modifying the code for stability 

In [13]:
def V_terminal(X, S):
    payoff = torch.clamp(X - call_payoff(S), min=-20, max=20)
    return -torch.exp(-gamma * payoff)

def hjb_residual(tau, X, S, v):
    V, V_tau, V_X, V_S, V_v, V_XX, V_SS, V_vv, V_XS, V_Xv, V_Sv = V_and_grads(tau, X, S, v)
    sqrtv = torch.sqrt(torch.clamp(v, min=1e-8))

    # Drift and diffusion terms
    drift_X = r * X * V_X
    drift_v = kappa * (theta - v) * V_v
    diff_S  = 0.5 * v * S**2 * V_SS
    diff_v  = 0.5 * xi**2 * v * V_vv
    mix_Sv  = rho * xi * S * sqrtv * V_Sv

    # Optimal control term — stabilize denominator
    num = (mu - r) * V_X + v * S * V_XS + rho * xi * v * V_Xv
    den = 2.0 * torch.sign(V_XX) * torch.clamp(torch.abs(v * V_XX), min=1e-3)
    control = (num**2) / den

    R = -V_tau + drift_X + drift_v + diff_S + diff_v + mix_Sv - control
    return R, V


train(steps=30000)


[   0] total=4.000e+34 pde=7.527e-01 term=4.000e+34 bc=4.365e-09
[ 200] total=3.985e+34 pde=6.994e-01 term=3.985e+34 bc=4.444e-09
[ 400] total=4.065e+34 pde=7.554e-01 term=4.065e+34 bc=4.367e-09
[ 600] total=4.028e+34 pde=7.181e-01 term=4.028e+34 bc=4.060e-09
[ 800] total=4.044e+34 pde=7.365e-01 term=4.044e+34 bc=4.555e-09
[1000] total=4.009e+34 pde=7.530e-01 term=4.009e+34 bc=4.484e-09
[1200] total=4.018e+34 pde=7.517e-01 term=4.018e+34 bc=3.940e-09
[1400] total=4.067e+34 pde=7.493e-01 term=4.067e+34 bc=4.265e-09
[1600] total=3.976e+34 pde=7.190e-01 term=3.976e+34 bc=4.225e-09
[1800] total=4.006e+34 pde=6.933e-01 term=4.006e+34 bc=4.603e-09
[2000] total=3.968e+34 pde=7.066e-01 term=3.968e+34 bc=4.137e-09
[2200] total=3.998e+34 pde=6.860e-01 term=3.998e+34 bc=4.147e-09
[2400] total=3.963e+34 pde=6.899e-01 term=3.963e+34 bc=4.287e-09
[2600] total=4.000e+34 pde=7.817e-01 term=4.000e+34 bc=4.405e-09
[2800] total=3.961e+34 pde=7.737e-01 term=3.961e+34 bc=4.342e-09
[3000] total=3.980e+34 pd

KeyboardInterrupt: 